In [1]:
import tensorflow as tf
import data_generator
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16
from keras_cv.losses import IoULoss

num_classes = 96

input_shape = (448, 448, 3)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)

bbox_output = Dense(4, activation='linear', name='bboxes_output')(x)

labels_output = Dense(num_classes, activation='softmax', name='labels_output')(x)

model = Model(inputs=base_model.input, outputs=[bbox_output, labels_output])

model.compile(optimizer='rmsprop', loss={'bboxes_output': IoULoss(), 'labels_output': 'categorical_crossentropy'}, metrics={'bboxes_output': 1 - IoULoss(), 'labels_output': 'accuracy'}, jit_compile = False)

print(model.summary())

2024-08-01 15:41:27.702062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 15:41:27.722660: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 15:41:27.729959: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-01 15:41:27.745190: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-01 15:41:29.280938: W tensorflow/compiler/tf2

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 21s 0us/step


2024-08-01 15:41:53.162488: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-08-01 15:41:53.242110: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-08-01 15:41:53.326819: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 448, 448,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 448, 448,  │      1,792 │ input_layer[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 448, 448,  │     36,928 │ block1_conv1[0][… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool         │ (None, 224, 224,  │          0 │ block1_conv2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 224, 224,  │     73,856 │ block1_pool[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2        │ (None, 224, 224,  │    147,584 │ block2_conv1[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 112, 112,  │          0 │ block2_conv2[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1        │ (None, 112, 112,  │    295,168 │ block2_pool[0][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2        │ (None, 112, 112,  │    590,080 │ block3_conv1[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv3        │ (None, 112, 112,  │    590,080 │ block3_conv2[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_pool         │ (None, 56, 56,    │          0 │ block3_conv3[0][… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv1        │ (None, 56, 56,    │  1,180,160 │ block3_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv2        │ (None, 56, 56,    │  2,359,808 │ block4_conv1[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv3        │ (None, 56, 56,    │  2,359,808 │ block4_conv2[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_pool         │ (None, 28, 28,    │          0 │ block4_conv3[0][… │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv1        │ (None, 28, 28,    │  2,359,808 │ block4_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv2        │ (None, 28, 28,    │  2,359,808 │ block5_conv1[0][

 Total params: 66,415,524 (253.36 MB)

 Trainable params: 51,697,764 (197.21 MB)

 Non-trainable params: 14,717,760 (56.14 MB)

None


In [5]:
image_dir_train = './output/train'
annotation_dir_train = './output/train'

image_dir_test = './output/test'
annotation_dir_test = './output/test'

batch_size = 4
epochs = 100

from collections import defaultdict
import os
data_path = './symbols.txt'
transforms = defaultdict()
with open(data_path, 'r') as txt_file:
    i = 0
    for line in txt_file:
        transforms[line.rstrip()] = i
        i += 1

train_generator = data_generator.DataGenerator(
    image_dir_train, annotation_dir_train,
    transforms=transforms,
    batch_size=batch_size,
    image_size=448
)

validation_generator = data_generator.DataGenerator(
    image_dir_test, annotation_dir_test,
    transforms=transforms,
    batch_size=batch_size,
    image_size=448
)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    # steps_per_epoch=len(train_generator),
    # validation_steps=len(validation_generator),
)

model.save_weights('./results/HCR-Net-Detection.h5')

Epoch 1/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 309s 3s/step - bboxes_output_iou_metric: 0.0000e+00 - labels_output_accuracy: 0.2009 - loss: 4.4675 - val_bboxes_output_iou_metric: 0.0000e+00 - val_labels_output_accuracy: 0.0000e+00 - val_loss: 6.1967
Epoch 2/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 324s 3s/step - bboxes_output_iou_metric: 0.0000e+00 - labels_output_accuracy: 0.3075 - loss: 3.9174 - val_bboxes_output_iou_metric: 0.0000e+00 - val_labels_output_accuracy: 0.0068 - val_loss: 6.1962
Epoch 3/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 343s 3s/step - bboxes_output_iou_metric: 0.0000e+00 - labels_output_accuracy: 0.4166 - loss: 3.4417 - val_bboxes_output_iou_metric: 0.0000e+00 - val_labels_output_accuracy: 0.0068 - val_loss: 6.3390
Epoch 4/100
100/100 ━━━━━━━━━━━━━━━━━━━━ 345s 3s/step - bboxes_output_iou_metric: 0.0000e+00 - labels_output_accuracy: 0.5548 - loss: 2.9603 - val_bboxes_output_iou_metric: 0.0000e+00 - val_labels_output_accuracy: 0.0203 - val_loss: 6.3986
Epoch 5/100
100/100 ━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [6]:
import matplotlib.pyplot as plt

plt.plot(history.history['labels_output_accuracy'])
plt.plot(history.history['val_labels_output_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['labels_output_loss'])
plt.plot(history.history['val_labels_output_loss'])
plt.title('Model Classification Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['bboxes_output_iou_metric'])
plt.plot(history.history['val_bboxes_output_iou_metric'])
plt.title('Model IoU Metric')
plt.ylabel('IoU')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['bboxes_output_loss'])
plt.plot(history.history['val_bboxes_output_loss'])
plt.title('Model Bounding Box Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

NameError: name 'history' is not defined